In [ ]:
# La Cam da OpenCV deve essere usate tramite GStreamer con un comando dedicato
# https://stackoverflow.com/questions/64272731/open-cv-shows-green-screen-on-jetson-nano
#
# Eseguire :
# sudo nvpmodel -m 0
# sudo jetson_clocks
# per portare a 4 le CPU operative. OpenCV su NVIDIA Jetson Nano non è ottimizzato per usare la GPU
# Il codice gira tutto su CPU con una certa lentezza.
#
import cv2
import numpy as np
import math
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, display, Image
import ipywidgets
import time

%matplotlib inline


In [ ]:
# make_chunks function creats chunks.
# inputs -- EdgeArray and the size_of_chunk to create.
# output -- yield successive n-sized chunks.
def make_chunks(EdgeArray, size_of_chunk):
    chunks = []
    for i in range(0, len(EdgeArray), size_of_chunk):
        chunks.append(EdgeArray[i:i + size_of_chunk])
    return chunks


In [ ]:
#
# https://github.com/JetsonHacksNano/CSI-Camera/blob/master/simple_camera.py
#

# MIT License
# Copyright (c) 2019-2022 JetsonHacks

# Using a CSI camera (such as the Raspberry Pi Version 2) connected to a
# NVIDIA Jetson Nano Developer Kit using OpenCV
# Drivers for the camera and OpenCV are included in the base image

import cv2

""" 
gstreamer_pipeline returns a GStreamer pipeline for capturing from the CSI camera
Flip the image by setting the flip_method (most common values: 0 and 2)
display_width and display_height determine the size of each camera pane in the window on the screen
Default 1920x1080 displayd in a 1/4 size window
"""

def gstreamer_pipeline(
    sensor_id=0,
    capture_width=640,
    capture_height=480,
    display_width=640,
    display_height=480,
    framerate=20,
    flip_method=0,
):
    return (
        "nvarguscamerasrc sensor-id=%d !"
        "video/x-raw(memory:NVMM), width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            sensor_id,
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

In [ ]:
#
# To display images to verify. Set to zero if not intrested in displaying images.
testmode = 1  

# Code to draw edge representation.

# cap = cv2.VideoCapture(0)  # Creating object to capturing frame from inbult camera(0) or the external camera(1)
cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)

if testmode == 1:
    display_handle=display(None, display_id=True)
    navigation_w = ipywidgets.Image(format='jpeg')

StepSize = 5

try:
    while (1):
        _, frame = cap.read()  # Reading the frame from the object.

        original_frame = frame.copy()  # Copy of frame which will be used for to compare with other images after appling various operations.
        img_edgerep = frame.copy()  # Copy of frame which will be used for edge representation.
        img_contour = frame.copy()  # Copy of frame which will be used for drawing contours.
        img_navigation = frame.copy()  # Copy of frame which will be used for indicating direction of navigation.

        blur = cv2.bilateralFilter(img_edgerep, 9, 40, 40)  # Blurring the image to remove the noise present in the image.
        edges = cv2.Canny(blur, 50, 100)  # Obtaining clear edges using canny edge detector.

        img_edgerep_h = img_edgerep.shape[0] - 1  # Storing the height of the image which will be used in for loop.
        img_edgerep_w = img_edgerep.shape[1] - 1  # Storing the width of the image which will be used in for loop.

        EdgeArray = []  # Initilizing the array to store the concerned edges for edge representation.

        for j in range(0, img_edgerep_w, StepSize):  # FOR loop along the width of the image with given stepsize.
            pixel = (j, 0)  # If no edge found in column this value will be stored in edgearray.
            for i in range(img_edgerep_h - 5, 0, -1):  # FOR loop along the height of the image.
                if edges.item(i, j) == 255:  # Checking for edges.
                    pixel = (j, i)
                    break  # If edge is found break and go for the next colomn.
            EdgeArray.append(pixel)  # Store the eged detected in EgdeArray.

        for x in range(len(
                EdgeArray) - 1):  # Joining each edge to diferentiate the frame into free space and conjusted space(with objects)
            cv2.line(img_edgerep, EdgeArray[x], EdgeArray[x + 1], (0, 255, 0), 1)

        for x in range(len(
                EdgeArray)):  # Joining each point in the EdgeArray to the respective bottom edge of the frame to represent free space for the bot to move around
            cv2.line(img_edgerep, (x * StepSize, img_edgerep_h), EdgeArray[x], (0, 255, 0), 1)

        # Code to draw contours.

        blurred_frame = cv2.bilateralFilter(img_contour, 9, 75, 75)
        gray = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 106, 255, 1)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(img_edgerep, contours, -1, (0, 0, 255), 3)

        # Code to decide direction of navigation

        number_of_chunks = 3
        size_of_chunk = int(len(EdgeArray) / number_of_chunks)
        chunks = make_chunks(EdgeArray, size_of_chunk)  # Calling make_chunks function to create the chunks.
        avg_of_chunk = []
        for i in range(len(chunks) - 1):
            x_vals = []
            y_vals = []
            for (x, y) in chunks[i]:  # Storing the x and y value saperatly to find average.
                x_vals.append(x)
                y_vals.append(y)
            avg_x = int(np.average(x_vals))
            avg_y = int(np.average(y_vals))
            avg_of_chunk.append([avg_y, avg_x])
            cv2.line(frame, (int(img_edgerep_w / 2), img_edgerep_h), (avg_x, avg_y), (255, 0, 0),
                         2)  # Draw lines to each average chunks to decide the direction of navigation.

        forwardEdge = avg_of_chunk[1]
        cv2.line(frame, (int(img_edgerep_w / 2), img_edgerep_h), (forwardEdge[1], forwardEdge[0]), (0, 255, 0), 3)
        farthest_point = (min(avg_of_chunk))
        # print(farthest_point)

        if forwardEdge[0] > 250:  # Checking for the object at the front is close to bot.
            if farthest_point[1] < 310:  # Checking for the farthest_point on the left of the frame.
                direction = "Move left "
                print(direction)
            else:
                direction = "Move right "
                print(direction)
        else:
            direction = "Move forward "
            print(direction)

            # Code to display the results
        if testmode == 1:
            font = cv2.FONT_HERSHEY_SIMPLEX
            navigation = cv2.putText(frame, direction, (275, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
            clear_output(wait=True)            
            lines, columns, _ =  navigation.shape
            navigation = cv2.resize(navigation, (int(columns/4), int(lines/4)))
            navigation_w.value =cv2.imencode('.jpeg', navigation)[1].tobytes()
            display(navigation_w)
            
            time.sleep(0.025)
            
except KeyboardInterrupt:
    pass
finally:
    cap.release()
    display_handle.update(None)    

In [ ]:
navigation.shape

In [ ]:
cv2.destroyAllWindows
cap.release()